In [1]:
import contextlib
import os
import sys
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from model.point_dataset import SCMDataset
from model.point_model import PointNetOneX
# import EarlyStopping
from model.pytorchtools import EarlyStopping
from tqdm import tqdm
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

In [2]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)# weight_decay=0.001)
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
    return optimizer, scheduler

In [3]:
def load_data(data_dir, concat): 
    train_set = SCMDataset(
        root=data_dir,
        concat = concat)
    
    val_set = SCMDataset(
        root=data_dir,
        split = 'val',
        concat = concat)

    test_set = SCMDataset(
        root=data_dir,
        split = 'test',
        concat = concat)
    
    return train_set, val_set, test_set 

In [4]:
def train(train_loader, model, optimizer, device, epoch):
    """ Train the model on num_steps batches 
    Args: 
        train_loader: a torch.utils.data.DataLoader object that fetches the data
        model: the neural network 
        optimizer: adams 
    """
    model.train()
    running_loss = 0.0
    num_batch = len(train_loader)

    for i, (points, target, _) in enumerate(train_loader): 
        points = points.transpose(2, 1) 
        points, target = points.to(device), target.to(device)

        # forward + backward + optimize 
        pred = model(points)
        loss = F.mse_loss(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics 
        running_loss += loss.item()
        # if i % 300 == 0: 
        #     print('[%d: %d/%d] train loss: %f lr = %f' % (epoch, i, num_batch, loss.item(), optimizer.param_groups[0]["lr"]))

    return running_loss / num_batch 

In [5]:
def validate(val_dataloader, model, device): 
    model.eval()
    val_running_loss = 0.0
    num_batch = len(val_dataloader)

    with torch.no_grad(): 
        for points, target, _ in val_dataloader:
            points = points.transpose(2, 1)
            points, target = points.to(device), target.to(device)
            # points, target = points.to(device), target.to(device)

            # outputs  = model(points, sys_scalar)
            outputs  = model(points)
            loss = F.mse_loss(outputs, target)

            val_running_loss += loss.item() 

    return val_running_loss / num_batch

In [6]:
def plot_pramas(test_y, test_pred, foldername, filename): 

    # print("R2 of training is: ", r2_score(train_y, train_pred))
    print("R2 of test is: ", r2_score(test_y, test_pred))

    np.savetxt(f'{foldername}/test_predict_{filename}.txt', test_pred)
    np.savetxt(f'{foldername}/test_target_{filename}.txt', test_y)

    test_mse = mean_squared_error(test_y, test_pred)
    test_mae = mean_absolute_error(test_y, test_pred)

    print('Test set results for %i samples:' % test_pred.shape[0])
    print('MSE:', test_mse)
    print('MAE:', test_mae)

In [7]:
def test_accuracy(net, testloader, foldername, filename, device): 
    test_pred = []
    test_y = [] 

    running_loss = 0
    with torch.no_grad():
        for data in testloader:
            points, target, _ = data
            points = points.transpose(2, 1) 
            points, target = points.to(device), target.to(device)
            outputs = net(points)
            loss = F.mse_loss(outputs, target)

            running_loss += loss.item() * points.size(0)

            pred_val_numpy = outputs.data.cpu().numpy()
            target_val_numpy = target.data.cpu().numpy()
            test_pred.append(pred_val_numpy)
            test_y.append(target_val_numpy)

    test_pred = np.concatenate(test_pred, axis=0)
    test_y = np.concatenate(test_y, axis=0)

    plot_pramas(test_y, test_pred, foldername, filename)
    
    print('MSE loss on test set is:', running_loss / len(testloader.dataset))    

In [8]:
def train_model(model, train_loader, val_loader, test_loader, optimizer, lr_scheduler, isSch, device, name, res_dir, patience = 20, n_epochs = 50): 
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = [] 

    blue = lambda x: '\033[94m' + x + '\033[0m'
    checkpoint_dir = os.path.join(res_dir, 'checkpoints')

    with contextlib.suppress(OSError):
        os.makedirs(res_dir)
        os.makedirs(checkpoint_dir)

    checkpoint_path = os.path.join(checkpoint_dir, f'{name}.pt')
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience, verbose=True, path = checkpoint_path)

    for epoch in tqdm(range(1, n_epochs + 1)):
        ###################
        # train the model #
        ###################
        train_epoch_loss = train(train_loader, model, optimizer, device, epoch)
        val_epoch_loss = validate(val_loader, model, device)

        if isSch:
            lr_scheduler.step(val_epoch_loss) # no step lr fixed LR = 0.001 

        # # print loss every epoch 
        # print('[%d] train loss: %f ' % (epoch, train_epoch_loss))
        # print('[%d] %s loss: %f' % (epoch, blue('validate'), val_epoch_loss))

        avg_train_losses.append(train_epoch_loss)
        avg_valid_losses.append(val_epoch_loss)

        # add early stopping 
        early_stopping(val_epoch_loss, model)
        if early_stopping.early_stop: 
            print("Early stopping")
            break 

    # load the last checkpoint with the best model
    model.load_state_dict(torch.load(checkpoint_path))

    np.savetxt(os.path.join(res_dir, f'train_loss_{name}.csv'), avg_train_losses)
    np.savetxt(os.path.join(res_dir, f'val_loss_{name}.csv'), avg_valid_losses)

    test_accuracy(model, test_loader, res_dir, name, device)

In [9]:
def train_main(config): 
    data_dir = 'datasets/train'

    # get dataset 
    train_set, test_set, val_set = load_data(data_dir, config['addGlobal'])

    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=0)

    val_loader = torch.utils.data.DataLoader(
            val_set,
            batch_size=int(config["batch_size"]),
            shuffle=True,
            num_workers=0)
    
    test_loader = torch.utils.data.DataLoader(
            test_set, 
            batch_size=int(config["batch_size"]), 
            shuffle=False, num_workers=0)
    
    print("Creating model")
    model = PointNetOneX(config["l1"], config["l2"], config["l3"], config["l4"], config["l5"], 
                  config["dropout"], config["isBN"], config['isLN'],config['constraint'], config['pooling'])
    print(model)

    # device = torch.device('cuda')
    # if torch.cuda.device_count() > 1:
    #     model = nn.DataParallel(model)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer, lr_scheduler = build_optimizer(model, config['optimizer'], config['lr'])
    res_dir = 'complexDNN/res/'
    
    name = f"{config['model']}{config['l1']}{config['l2']}{config['l3']}{config['l4']}{config['l5']}lr{config['lr']}BS{config['batch_size']}isB{config['isBN']}ln{config['isLN']}cons{config['constraint']}Opt{config['optimizer']}sch{config['lr_scheduler']}pool{config['pooling']}addG{config['addGlobal']}"
    train_model(model, train_loader, val_loader, test_loader, optimizer, lr_scheduler, config['lr_scheduler'], device, name, res_dir, 40, 1000)

In [10]:
config = {"l1": 512,"l2": 512, "l3": 512,"l4": 256,"l5": 128,"dropout" : 0, "lr": 0.01,  "batch_size": 128, "model" : "PointNetOneX", "isBN": False,
        "isLN": True, "lr_scheduler": True, "addGlobal" : False, "pooling" : 'min', "constraint" : False,"optimizer": 'adam'}

In [11]:
import time 
start_time = time.perf_counter()
train_main(config) 
end_time = time.perf_counter() 
print('time used to train model with 40/1000 patience is: ', (end_time - start_time)/60, 'mins')

Creating model
PointNetOneX(
  (conv1): Conv1d(6, 512, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
  (conv3): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ln1): LayerNorm((512, 56), eps=1e-05, elementwise_affine=True)
  (ln2): LayerNorm((512, 56), eps=1e-05, elementwise_affine=True)
  (ln3): LayerNorm((512, 56), eps=1e-05, elementwise_affine=True)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=5, bias=True)
  (dropout): Dropout(p=0, inplace=False)
  (bn4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): Batc

  0%|▏                                                                                                                                 | 1/1000 [00:05<1:29:20,  5.37s/it]

Validation loss decreased (inf --> 0.030042).  Saving model ...


  0%|▎                                                                                                                                 | 2/1000 [00:09<1:19:09,  4.76s/it]

Validation loss decreased (0.030042 --> 0.017884).  Saving model ...


...

 22%|████████████████████████████▋                                                                                                     | 221/1000 [16:20<57:04,  4.40s/it]

EarlyStopping counter: 39 out of 40


 22%|████████████████████████████▋                                                                                                     | 221/1000 [16:24<57:51,  4.46s/it]

EarlyStopping counter: 40 out of 40
Early stopping


R2 of test is:  0.9375733529381579
Test set results for 11309 samples:
MSE: 0.0038909372
MAE: 0.03546797
MSE loss on test set is: 0.003890940064495336
time used to train model with 40/1000 patience is:  16.440725128368165 mins


## prediction on training or experiment data

In [14]:
def predict_on_test(config): 
    
    data_dir = 'datasets/train'
    exp_cscl_dir = 'datasets/exp/rutitle_cscl'
    exp_licl_dir = 'datasets/exp/rutitle_LiCl'

    # get training dataset 
    train_set, _, _ = load_data(exp_cscl_dir, config['addGlobal'])
    # train_set, _, _ = load_data(exp_licl_dir, config['addGlobal'])

    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=int(config["batch_size"]),
        shuffle=True)
            
    print("Creating model")
    model = PointNetOneX(config["l1"], config["l2"], config["l3"], config["l4"], config["l5"], 
                  config["dropout"], config["isBN"], config['isLN'],config['constraint'], config['pooling'])
    print('created model is: ', model)
    
    name = f"{config['model']}{config['l1']}{config['l2']}{config['l3']}{config['l4']}{config['l5']}lr{config['lr']}BS{config['batch_size']}isB{config['isBN']}ln{config['isLN']}cons{config['constraint']}Opt{config['optimizer']}sch{config['lr_scheduler']}pool{config['pooling']}addG{config['addGlobal']}"
    name1 = f"cscl{config['model']}{config['l1']}{config['l2']}{config['l3']}{config['l4']}{config['l5']}lr{config['lr']}BS{config['batch_size']}isB{config['isBN']}ln{config['isLN']}cons{config['constraint']}Opt{config['optimizer']}sch{config['lr_scheduler']}pool{config['pooling']}addG{config['addGlobal']}"
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # device = torch.device("cuda")
    # if torch.cuda.device_count() > 1:
    #     model = nn.DataParallel(model)
    model.to(device)

    optimizer, lr_scheduler = build_optimizer(model, config['optimizer'], config['lr'])
    model.to(device)

    res_dir = 'complexDNN/res/'
    checkpoint_dir = os.path.join(res_dir, 'checkpoints')
    checkpoint_path = os.path.join(checkpoint_dir, f'{name}.pt')

    model.load_state_dict(torch.load(checkpoint_path))

    # test on test set 
    test_accuracy(model, train_loader, res_dir, name1, device)   

In [15]:
import time 
start_time = time.perf_counter()
predict_on_test(config)
end_time = time.perf_counter() 
print('time used to train model with 20/250 patience is: ', (end_time - start_time), 'secs')

Creating model
created model is:  PointNetOneX(
  (conv1): Conv1d(6, 512, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
  (conv3): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ln1): LayerNorm((512, 56), eps=1e-05, elementwise_affine=True)
  (ln2): LayerNorm((512, 56), eps=1e-05, elementwise_affine=True)
  (ln3): LayerNorm((512, 56), eps=1e-05, elementwise_affine=True)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=5, bias=True)
  (dropout): Dropout(p=0, inplace=False)
  (bn4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

/home/chunhui/ENTER/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
